# Create basic visualisation elements using Plotly on DASH 

In [ ]:
https://plotly.com/python/plotly-express/

#https://dash.gallery/Portal/

In [1]:
# Import Libraries

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from sklearn import datasets, linear_model
from sklearn.metrics import r2_score

### Basic Visualisations with Time Series

In [2]:
# Import data 

df = pd.read_csv('visdf.csv')

In [10]:
column_data_types = df.dtypes
print("Data Types of DataFrame Columns:")
print(column_data_types)

Data Types of DataFrame Columns:
Comment    object
Time       object
Score       int64
ID          int64
dtype: object


In [11]:
df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d %H:%M:%S')

In [12]:
df.head()

,Comment,Time,Score,ID
0,Posting a bit early so people know where to go...,2023-10-10 11:14:12,1,1
1,20% fare reduction on public transport continu...,2023-10-10 13:55:05,39,2
2,This is good.,2023-10-10 14:16:50,3,3
3,"Aaaaaahhhhh, what have the Greens done for us!",2023-10-10 16:56:05,2,4
4,"€67 million to provide 16,000 craft apprentice...",2023-10-10 14:18:04,25,5


In [39]:
# Remove single outlier where 'ID' = 402

df = df[df['ID'] != 402].copy()

In [43]:
df.to_csv('datavisnew.csv', index=False)

In [40]:
# Test visualisation 1 - Line chart

fig = px.line(df, x='Time', y='Score', title='Score Over Time')
fig.show()


In [41]:
# Test visualisation 2 - Scatter



fig = px.scatter(df, x='Time', y='Score', color='Score', title='Score Over Time by ID')
fig.show()


In [45]:
# Final Time Series Plot

import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd



# Create Dash app
app = dash.Dash(__name__)

# Slider mark labels (hourly)
marks = {
    i: {'label': pd.to_datetime(i, unit='s').strftime('%H') + 'hr', 'style': {'transform': 'rotate(-90deg)', 'margin-left': '-10px'}}
    for i in range(int(df['Time'].min().timestamp()), int(df['Time'].max().timestamp()) + 1, 3600)
}

# Layout App
app.layout = html.Div([
    html.H1("Comment Scores Over Time"),
    
    # Add sliders
    dcc.RangeSlider(
        id='time-slider',
        marks=marks,
        min=df['Time'].min().timestamp(),
        max=df['Time'].max().timestamp(),
        step=3600,  
        value=[df['Time'].min().timestamp(), df['Time'].max().timestamp()]
    ),

    
    dcc.Graph(
        id='line-chart',
        config={'displayModeBar': False}  
    )
])

# Define callback
@app.callback(
    Output('line-chart', 'figure'),
    [Input('time-slider', 'value')]
)
def update_chart(selected_time_range):
    start_time, end_time = selected_time_range
    filtered_df = df[(df['Time'] >= pd.to_datetime(start_time, unit='s')) & (df['Time'] <= pd.to_datetime(end_time, unit='s'))]

    # Add color scale using 'Score'
    fig = px.scatter(filtered_df, x='Time', y='Score', color='Score', title='Score Over Time (Zoomed In)', color_continuous_scale='Magma', opacity=0.6)

    # Update color bar position to the right
    fig.update_layout(coloraxis_colorbar=dict(yanchor="middle", y=0.5, x=-0.25))

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
